# 🎯 GeoProj - Corrección de Distorsión Geométrica con Deep Learning

<div align="center">

![Python](https://img.shields.io/badge/Python-3.7+-blue.svg)
![PyTorch](https://img.shields.io/badge/PyTorch-1.9+-orange.svg)
![License](https://img.shields.io/badge/License-MIT-green.svg)

</div>

Este notebook implementa el método de **Blind Geometric Distortion Correction** usando Deep Learning para detectar y corregir automáticamente distorsiones geométricas en imágenes.

---

## 📖 Descripción

El modelo utiliza una arquitectura encoder-decoder con clasificación para:
1. **Detectar** el tipo de distorsión presente en la imagen
2. **Calcular** un campo de flujo óptico para la corrección
3. **Aplicar** la rectificación para obtener la imagen corregida

**Paper:** Li, X., Zhang, B., Sander, P. V., & Liao, J. (2019)  
*Blind Geometric Distortion Correction on Images Through Deep Learning*  
IEEE/CVF CVPR 2019

**Repositorio (Versión Mejorada):** https://github.com/gallobruno81/GeoProj  
**Repositorio Original:** https://github.com/xiaoyu258/GeoProj

---

## 🎯 Tipos de Distorsión Soportados

| Tipo | Descripción | Ejemplo Común |
|------|-------------|---------------|
| 🔴 **Barrel** | Distorsión de barril | Lentes gran angular, cámaras de acción |
| 🔵 **Pincushion** | Distorsión de cojín | Lentes telefoto, zoom |
| 🔄 **Rotation** | Rotación de imagen | Inclinación de cámara |
| ✂️ **Shear** | Cizallamiento | Escaneo angular |
| 📐 **Projective** | Distorsión proyectiva | Perspectiva incorrecta |
| 🌊 **Wave** | Ondulación | Distorsiones complejas |

---

## 🚀 Inicio Rápido

1. **Configura GPU** → `Runtime` > `Change runtime type` > `GPU`
2. **Ejecuta Celdas 2-5** → Instala dependencias y clona repositorio
3. **Sube Modelos** → Celda 7 (descarga primero desde Google Drive)
4. **Sube Imagen** → Celda 9
5. **Procesa** → Ejecuta Celdas 11-18

**⏱️ Tiempo estimado:** 2-5 minutos por imagen

**💡 Consejo:** Después de la primera imagen, solo repite las Celdas 9-18

---

## 📋 Requisitos

- Google Colab (con GPU recomendado)
- Modelos pre-entrenados ([descargar aquí](https://drive.google.com/open?id=1Tdi92IMA-rrX2ozdUMvfiN0jCZY7wIp_))
- Imagen con distorsión geométrica

---

¡Comencemos! 👇


## 1️⃣ Configuración Inicial

### Verificar y Configurar GPU


In [ ]:
import torch
import subprocess

# Verificar GPU
print("="*60)
print("INFORMACIÓN DEL SISTEMA")
print("="*60)

try:
    subprocess.run(['nvidia-smi'], check=True)
    print("\n✓ GPU NVIDIA detectada")
except:
    print("\n⚠️ No se detectó GPU NVIDIA")

print("\n" + "="*60)
print(f"PyTorch versión: {torch.__version__ if 'torch' in dir() else 'No instalado'}")
print(f"CUDA disponible: {torch.cuda.is_available() if 'torch' in dir() else 'N/A'}")
if torch.cuda.is_available():
    print(f"CUDA versión: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
print("="*60)


In [ ]:
# Instalar dependencias
print("Instalando dependencias necesarias...")
print("Esto puede tomar 1-2 minutos...\n")

!pip install torch>=1.9.0 torchvision>=0.10.0 --quiet
!pip install numpy>=1.19.0 --quiet
!pip install scikit-image>=0.18.0 scipy>=1.7.0 --quiet
!pip install numba>=0.54.0 --quiet
!pip install matplotlib>=3.3.0 Pillow>=8.3.0 --quiet

print("\n" + "="*60)
print("✓ Todas las dependencias instaladas correctamente")
print("="*60)

# Verificar instalación
import torch
import torchvision
import numpy
import skimage
import scipy
import numba
import matplotlib
from PIL import Image

print("\nVersiones instaladas:")
print(f"  - torch: {torch.__version__}")
print(f"  - torchvision: {torchvision.__version__}")
print(f"  - numpy: {numpy.__version__}")
print(f"  - scikit-image: {skimage.__version__}")
print(f"  - scipy: {scipy.__version__}")
print(f"  - numba: {numba.__version__}")
print(f"  - matplotlib: {matplotlib.__version__}")
print(f"  - Pillow: {Image.__version__}")


## 2️⃣ Clonar Repositorio


In [ ]:
import os
import shutil

# Repositorio mejorado con inference.py y rectify.py
REPO_URL = "https://github.com/gallobruno81/GeoProj.git"

print("="*60)
print("CLONANDO REPOSITORIO")
print("="*60)
print("\n📦 Repositorio mejorado con scripts de inferencia y rectificación")

# Asegurarse de estar en /content
%cd /content

# Si existe un GeoProj viejo, eliminarlo
if os.path.exists('GeoProj'):
    print("\n🗑️ Eliminando repositorio anterior...")
    shutil.rmtree('GeoProj')

# Clonar repositorio
print(f"\n🔄 Clonando desde: {REPO_URL}")
!git clone {REPO_URL}

# Cambiar al directorio del proyecto
%cd GeoProj

print(f"\n📁 Directorio actual: {os.getcwd()}")

# Verificar archivos clave
print("\n✅ Verificando archivos del proyecto:")
files_to_check = ['inference.py', 'rectify.py', 'modelNetM.py', 'eval.py']
for f in files_to_check:
    status = "✓" if os.path.exists(f) else "✗"
    print(f"  {status} {f}")

if not os.path.exists('inference.py'):
    print("\n❌ ERROR: inference.py no encontrado!")
    print("Puede que se haya clonado el repositorio incorrecto.")
else:
    print("\n✅ Repositorio clonado correctamente!")
    print("="*60)


## 3️⃣ Descargar Modelos Pre-entrenados

**IMPORTANTE:** Descarga los modelos manualmente desde:  
https://drive.google.com/open?id=1Tdi92IMA-rrX2ozdUMvfiN0jCZY7wIp_

Luego súbelos usando la celda siguiente.


In [ ]:
import os
import shutil

os.makedirs('models', exist_ok=True)

print("="*60)
print("CONFIGURANDO MODELOS")
print("="*60)

# Buscar modelos en /kaggle/input/ (datasets de Kaggle)
input_dir = '/kaggle/input'
model_files = ['model_en.pkl', 'model_de.pkl', 'model_class.pkl']
found_models = []

print("\n🔍 Buscando modelos en datasets de Kaggle...")

if os.path.exists(input_dir):
    for root, dirs, files in os.walk(input_dir):
        for model_file in model_files:
            if model_file in files:
                source = os.path.join(root, model_file)
                dest = os.path.join('models', model_file)
                shutil.copy2(source, dest)
                found_models.append(model_file)
                print(f"  ✓ {model_file} copiado desde dataset")

# Verificar
print("\n" + "="*60)
print("VERIFICANDO MODELOS")
print("="*60)
!ls -lh models/

missing = [f for f in model_files if f not in found_models]

if not missing:
    print("\n✅ Todos los modelos disponibles!")
else:
    print(f"\n⚠️ FALTAN: {', '.join(missing)}")
    print("\n💡 Agrega el dataset con los modelos:")
    print("  1. Click '+ Add Data' (panel derecho)")
    print("  2. Busca tu dataset con los .pkl")
    print("  3. Click 'Add'")
    print("  4. Vuelve a ejecutar esta celda")

print("="*60)


## 4️⃣ Subir Imagen para Procesar


In [ ]:
from IPython.display import Image as IPImage, display
from PIL import Image
import os
import shutil

print("="*60)
print("CONFIGURANDO IMAGEN")
print("="*60)

# Buscar imagen en /kaggle/input/ (dataset de Kaggle)
input_image = None
print("\n🔍 Buscando imagen en datasets de Kaggle...")

for root, dirs, files in os.walk('/kaggle/input'):
    for file in files:
        if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            source = os.path.join(root, file)
            # Copiar a directorio de trabajo
            shutil.copy2(source, file)
            input_image = file
            print(f"  ✓ {file} encontrada y copiada")
            break
    if input_image:
        break

if input_image:
    img = Image.open(input_image)
    width, height = img.size
    mode = img.mode
    
    print("\n" + "="*60)
    print("INFORMACIÓN DE LA IMAGEN")
    print("="*60)
    print(f"✓ Archivo: {input_image}")
    print(f"  - Dimensiones: {width}x{height} px")
    print(f"  - Modo de color: {mode}")
    print(f"  - Tamaño: {os.path.getsize(input_image)/1024:.1f} KB")
    print("="*60)
    
    print("\n📷 Vista previa:")
    display(IPImage(filename=input_image, width=500))
else:
    print("\n⚠️ No se encontró ninguna imagen")
    print("\n💡 Agrega una imagen al dataset:")
    print("  1. Click '+ Add Data'")
    print("  2. Sube tu imagen o usa el dataset existente")
    print("  3. Vuelve a ejecutar esta celda")


## 5️⃣ Procesar Imagen (Detección + Flow)


In [ ]:
import time
import os

# Verificar que inference.py existe
if not os.path.exists('inference.py'):
    print("❌ ERROR: inference.py no encontrado!")
    print("Por favor, ejecuta la Celda 4 (Clonar Repositorio).")
    raise FileNotFoundError("inference.py no encontrado")

print(f"📁 Directorio de trabajo: {os.getcwd()}\n")

print("="*60)
print("INICIANDO PROCESAMIENTO")
print("="*60)
print("\n⏳ Detectando tipo de distorsión y calculando flujo óptico...")
print("   (esto puede tomar 10-30 segundos)\n")

start_time = time.time()

# Ejecutar inferencia (escapar nombre de archivo con espacios/caracteres especiales)
!python inference.py --input_image "{input_image}" --output_dir results --model_dir models

elapsed_time = time.time() - start_time

print("\n" + "="*60)
print("✅ PROCESAMIENTO COMPLETADO")
print("="*60)
print(f"⏱️ Tiempo transcurrido: {elapsed_time:.1f} segundos")
print("="*60)


In [ ]:
from IPython.display import Image as IPImage, display
import os

base_name = os.path.splitext(input_image)[0]
flow_viz_path = f'results/{base_name}_flow_viz.png'

print("="*60)
print("VISUALIZACIÓN DEL FLUJO ÓPTICO")
print("="*60)
print("\n📊 El flujo óptico representa la corrección que se aplicará:")
print("   - Rojo/Azul: dirección del desplazamiento")
print("   - Intensidad: magnitud de la corrección\n")

if os.path.exists(flow_viz_path):
    display(IPImage(filename=flow_viz_path, width=900))
    print("\n✓ Visualización generada correctamente")
else:
    print("⚠️ No se encontró la visualización del flujo")
    print("   Verifica que el procesamiento anterior se completó exitosamente.")


## 6️⃣ Aplicar Corrección (Rectificación)

**Nota:** Esto requiere GPU con CUDA. Si no funciona en Colab, el flujo ya está calculado.


In [ ]:
import os
import time

base_name = os.path.splitext(input_image)[0]
resized_path = f'results/{base_name}_resized.jpg'
flow_path = f'results/{base_name}_flow.npy'

print(f"📁 Directorio de trabajo: {os.getcwd()}\n")

print("="*60)
print("APLICANDO RECTIFICACIÓN")
print("="*60)
print("\n⚙️ Aplicando la corrección de distorsión...")
print("   (esto requiere GPU con CUDA)\n")

start_time = time.time()
rectification_success = False

try:
    # Intentar rectificación (escapar nombres con espacios)
    # Nota: output_dir por defecto es 'results'
    !python rectify.py --img_path "{resized_path}" --flow_path "{flow_path}"
    rectification_success = True
    elapsed_time = time.time() - start_time
    
    print("\n" + "="*60)
    print("✅ RECTIFICACIÓN COMPLETADA")
    print("="*60)
    print(f"⏱️ Tiempo: {elapsed_time:.1f} segundos")
    print("="*60)
    
except Exception as e:
    print("\n" + "="*60)
    print("⚠️ ERROR EN RECTIFICACIÓN")
    print("="*60)
    print(f"Error: {e}")
    print("\n💡 SOLUCIÓN:")
    print("   1. El flujo óptico ya fue calculado y guardado")
    print("   2. Descarga el archivo 'results.zip' al final")
    print("   3. Usa rectify.py localmente con una GPU CUDA")
    print("="*60)
    rectification_success = False


## 7️⃣ Ver Resultados


In [ ]:
from IPython.display import Image as IPImage, display
import os

base_name = os.path.splitext(input_image)[0]

print("="*60)
print("RESULTADOS FINALES")
print("="*60)

# Mostrar comparación si existe
comparison_path = f'results/{base_name}_comparison.png'
corrected_path = f'results/{base_name}_corrected.png'

if os.path.exists(comparison_path):
    print("\n🎨 Comparación Antes/Después:\n")
    display(IPImage(filename=comparison_path, width=1000))
    print("\n✅ Corrección aplicada exitosamente!")
elif os.path.exists(corrected_path):
    print("\n🎨 Imagen Corregida:\n")
    display(IPImage(filename=corrected_path, width=500))
    print("\n✅ Imagen corregida generada!")
else:
    print("\n⚠️ No se encontraron imágenes corregidas")
    print("\nArchivos disponibles en results/:")
    !ls -lh results/
    
print("\n" + "="*60)
print("Archivos generados:")
!ls -lh results/ 2>/dev/null | grep "{base_name}"
print("="*60)


## 8️⃣ Descargar Resultados


In [ ]:
import os

print("="*60)
print("DESCARGAR RESULTADOS")
print("="*60)
print("\n📦 Comprimiendo resultados...")

# Comprimir resultados
!zip -r -q results.zip results/

# Obtener tamaño del zip
zip_size = os.path.getsize('results.zip') / 1024 / 1024  # MB

print(f"\n✓ Archivo comprimido: results.zip ({zip_size:.2f} MB)")
print("\nContenido del archivo:")
!zipinfo results.zip | head -20

print("\n" + "="*60)
print("⬇️ Iniciando descarga...")
print("="*60)

print("\n" + "="*60)
print("📥 DESCARGAR RESULTADOS EN KAGGLE")
print("="*60)
print("\nPara descargar los resultados:")
print("  1. Click 'Save Version' (arriba derecha)")
print("  2. Espera a que termine la ejecución")
print("  3. Ve a la tab 'Output'")
print("  4. Descarga 'results.zip'")
print("\nO descarga archivos individuales desde:")
print(f"  {os.getcwd()}/results/")
print("="*60)

print("\n📂 Archivos generados:")
print("   - *_flow.npy: flujo óptico calculado")
print("   - *_flow_viz.png: visualización del flujo")
print("   - *_resized.jpg: imagen redimensionada")
print("   - *_corrected.png: imagen corregida")
print("   - *_comparison.png: comparación antes/después")
print("   - *_mask.png: máscara de validez")


---

## 🚀 ¿Quieres Procesar Otra Imagen?

Para procesar otra imagen, simplemente:
1. Vuelve a ejecutar la **Celda 9** (Subir Imagen)
2. Luego ejecuta las **Celdas 11-18** en orden

No necesitas volver a:
- Instalar dependencias (Celda 3)
- Clonar el repositorio (Celda 5) 
- Subir los modelos (Celda 7)

---

## 📝 Notas Importantes

### Requisitos del Sistema

1. **GPU:** Se recomienda usar GPU para mejor rendimiento
   - En Colab: `Runtime` → `Change runtime type` → `GPU` (T4, V100, etc.)
   - La inferencia funciona en CPU, pero es ~10x más lenta

2. **CUDA para rectificación:** La etapa de rectificación requiere:
   - GPU con CUDA habilitada
   - numba con soporte CUDA
   - Si falla, descarga el flujo y procesa localmente

### Tamaños de Imagen

- **Entrada:** Cualquier resolución (se redimensiona automáticamente)
- **Procesamiento:** 256x256 (requisito del modelo)
- **Salida:** 256x256 (puedes escalar después si necesitas)

### Archivos Generados

| Archivo | Descripción |
|---------|-------------|
| `*_flow.npy` | Flujo óptico calculado (array NumPy) |
| `*_flow_viz.png` | Visualización del flujo (3 vistas) |
| `*_resized.jpg` | Imagen redimensionada a 256x256 |
| `*_corrected.png` | Imagen corregida final |
| `*_comparison.png` | Comparación antes/después |
| `*_mask.png` | Máscara de validez de píxeles |

### Tipos de Distorsión Soportados

✅ **Barrel** (Barril) - Lentes gran angular  
✅ **Pincushion** (Cojín) - Lentes telefoto  
✅ **Rotation** (Rotación) - Rotación de cámara  
✅ **Shear** (Cizallamiento) - Distorsión de corte  
✅ **Projective** (Proyectiva) - Perspectiva incorrecta  
✅ **Wave** (Onda) - Distorsiones complejas  

---

## 🔧 Troubleshooting

### Error: "Model files not found"
- Asegúrate de descargar los 3 archivos .pkl del Google Drive
- Verifica que los subiste correctamente en la Celda 7

### Error: "CUDA not available"
- Cambia el runtime: `Runtime` → `Change runtime type` → `GPU`
- Reinicia el runtime después de cambiar

### Error: "inference.py not found"
- Verifica que se clonó el repositorio correcto (gallobruno81/GeoProj)
- Vuelve a ejecutar la Celda 5 (eliminará el repo anterior y clonará el correcto)
- El repositorio original (xiaoyu258) NO tiene inference.py

### Error en rectificación (numba-cuda)
- Es normal en algunos entornos de Colab
- El flujo ya está calculado, descárgalo y procesa localmente
- Alternativamente, usa una instancia Colab Pro con mejor GPU

### Imagen muy grande o muy pequeña
- El modelo funciona mejor con imágenes de tamaño razonable (>256px)
- Las imágenes muy grandes se redimensionan automáticamente

### Resultados no satisfactorios
- Verifica que la imagen tenga distorsión geométrica real
- El modelo está entrenado para tipos específicos de distorsión
- Distorsiones muy extremas pueden no corregirse completamente

---

## 💡 Tips para Mejores Resultados

1. **Calidad de entrada:** Usa imágenes con buena iluminación y contraste
2. **Formato:** JPG o PNG funcionan bien
3. **Contenido:** Imágenes con líneas rectas (edificios, documentos) muestran mejor la corrección
4. **Procesamiento local:** Para producción, descarga el código y ejecuta localmente con GPU

---

## 📚 Referencias

**Paper:** Li, X., Zhang, B., Sander, P. V., & Liao, J. (2019).  
*Blind Geometric Distortion Correction on Images Through Deep Learning.*  
IEEE/CVF Conference on Computer Vision and Pattern Recognition (CVPR) 2019.

**Enlaces:**
- 📄 ArXiv: https://arxiv.org/abs/1909.03459  
- 💻 GitHub: https://github.com/gallobruno81/GeoProj (Versión mejorada)
- 💻 GitHub: https://github.com/xiaoyu258/GeoProj (Original)
- 🔗 Paper: http://openaccess.thecvf.com/content_CVPR_2019/html/Li_Blind_Geometric_Distortion_Correction_on_Images_Through_Deep_Learning_CVPR_2019_paper.html

---

## 🎓 Citación

Si usas este código en tu investigación, por favor cita:

```bibtex
@inproceedings{li2019blind,
  title={Blind Geometric Distortion Correction on Images Through Deep Learning},
  author={Li, Xiaoyu and Zhang, Bo and Sander, Pedro V and Liao, Jing},
  booktitle={Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition},
  pages={4855--4864},
  year={2019}
}
```

---

## ⭐ ¿Te fue útil?

Si este notebook te ayudó, considera:
- ⭐ Dar una estrella al repositorio en GitHub
- 🐛 Reportar bugs o problemas
- 💬 Compartir tus resultados
- 🤝 Contribuir con mejoras

---

**Desarrollado con ❤️ para la comunidad de Computer Vision**

*Última actualización: Octubre 2025*
